In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
from sklearn.metrics import plot_precision_recall_curve
import numpy as np
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("testdata.manual.2009.06.14.csv", sep=",")

In [3]:
df.head()

,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,"@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right."
0,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
1,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
2,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
3,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
4,4,8,Mon May 11 03:22:00 UTC 2009,kindle2,GeorgeVHulme,@richardebaker no. it is too big. I'm quite ha...


In [4]:
df.columns=[0, 1, 2, 3, 4, 5]

In [5]:
df.head(5)

,0,1,2,3,4,5
0,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
1,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
2,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
3,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
4,4,8,Mon May 11 03:22:00 UTC 2009,kindle2,GeorgeVHulme,@richardebaker no. it is too big. I'm quite ha...


In [6]:
df.shape

(497, 6)

In [7]:
df.describe

<bound method NDFrame.describe of      0      1                             2        3                4  \
0    4      4  Mon May 11 03:18:03 UTC 2009  kindle2           vcu451   
1    4      5  Mon May 11 03:18:54 UTC 2009  kindle2           chadfu   
2    4      6  Mon May 11 03:19:04 UTC 2009  kindle2            SIX15   
3    4      7  Mon May 11 03:21:41 UTC 2009  kindle2         yamarama   
4    4      8  Mon May 11 03:22:00 UTC 2009  kindle2     GeorgeVHulme   
..  ..    ...                           ...      ...              ...   
492  2  14072  Sun Jun 14 04:31:43 UTC 2009    latex          proggit   
493  0  14073  Sun Jun 14 04:32:17 UTC 2009    latex           sam33r   
494  4  14074  Sun Jun 14 04:36:34 UTC 2009    latex  iamtheonlyjosie   
495  0  14075  Sun Jun 14 21:36:07 UTC 2009     iran        plutopup7   
496  0  14076  Sun Jun 14 21:36:17 UTC 2009     iran     captain_pete   

                                                     5  
0    Reading my kindle2...  Love

In [8]:
df_new = df.drop([1, 2, 3, 4], axis=1)

In [9]:
df_new

,0,5
0,4,Reading my kindle2... Love it... Lee childs i...
1,4,"Ok, first assesment of the #kindle2 ...it fuck..."
2,4,@kenburbary You'll love your Kindle2. I've had...
3,4,@mikefish Fair enough. But i have the Kindle2...
4,4,@richardebaker no. it is too big. I'm quite ha...
...,...,...
492,2,Ask Programming: LaTeX or InDesign?: submitted...
493,0,"On that note, I hate Word. I hate Pages. I hat..."
494,4,Ahhh... back in a *real* text editing environm...
495,0,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."


In [10]:
df[0].value_counts()

4    181
0    177
2    139
Name: 0, dtype: int64

In [11]:
for c in df[df[0] == 4][5].head(5):
    print(c)

Reading my kindle2...  Love it... Lee childs is good read.
Ok, first assesment of the #kindle2 ...it fucking rocks!!!
@kenburbary You'll love your Kindle2. I've had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)
@mikefish  Fair enough. But i have the Kindle2 and I think it's perfect  :)
@richardebaker no. it is too big. I'm quite happy with the Kindle2.


In [12]:
for c in df[df[0] == 2][5].head(5):
    print(c)

Check this video out -- President Obama at the White House Correspondents' Dinner http://bit.ly/IMXUM
need suggestions for a good IR filter for my canon 40D ... got some? pls DM
@surfit: I just checked my google for my business- blip shows up as the second entry! Huh. Is that a good or ba... ? http://blip.fm/~6emhv
is in San Francisco at Bay to Breakers.
just landed at San Francisco


In [13]:
for c in df[df[0] == 0][5].head(5):
    print(c)

Fuck this economy. I hate aig and their non loan given asses.
@Karoli I firmly believe that Obama/Pelosi have ZERO desire to be civil.  It's a charade and a slogan, but they want to destroy conservatism
dear nike, stop with the flywire. that shit is a waste of science. and ugly. love, @vincentx24x
I was talking to this guy last night and he was telling me that he is a die hard Spurs fan.  He also told me that he hates LeBron James.
@ludajuice Lebron is a Beast, but I'm still cheering 4 the A..til the end.


In [14]:
train_df, test_df = train_test_split(df, test_size=20)

In [15]:
sentence_example = df.iloc[1][5]
tokens = word_tokenize(sentence_example, language="english")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("english")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="english")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]

In [16]:
print(f"Исходный текст: {sentence_example}")
print("-----------------")
print(f"Токены: {tokens}")
print("-----------------")
print(f"Токены без пунктуации: {tokens_without_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}")
print("-----------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("-----------------")

Исходный текст: Ok, first assesment of the #kindle2 ...it fucking rocks!!!
-----------------
Токены: ['Ok', ',', 'first', 'assesment', 'of', 'the', '#', 'kindle2', '...', 'it', 'fucking', 'rocks', '!', '!', '!']
-----------------
Токены без пунктуации: ['Ok', 'first', 'assesment', 'of', 'the', 'kindle2', '...', 'it', 'fucking', 'rocks']
-----------------
Токены без пунктуации и стоп слов: ['Ok', 'first', 'assesment', 'kindle2', '...', 'fucking', 'rocks']
-----------------
Токены после стемминга: ['ok', 'first', 'asses', 'kindle2', '...', 'fuck', 'rock']
-----------------


In [17]:
snowball = SnowballStemmer(language="english")
english_stop_words = stopwords.words("english")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="english")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in english_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [18]:
tokenize_sentence(sentence_example)

['ok', 'first', 'asses', 'kindle2', '...', 'fuck', 'rock']

In [19]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))

In [20]:
features = vectorizer.fit_transform(train_df[5])

In [21]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df[0])

LogisticRegression(random_state=0)

In [22]:
model.predict(features[0])

array([0], dtype=int64)

In [23]:
train_df[5].iloc[0]

'NOOOOOOO my DVR just died and I was only half way through the EA presser. Hate you Time Warner'

In [24]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ("model", LogisticRegression(random_state=0))
]
)

In [25]:
model_pipeline.fit(train_df[5], train_df[0])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x000001EA8899DF70>)),
                ('model', LogisticRegression(random_state=0))])

In [26]:
model_pipeline.predict(["Hello, are you okay?"])

array([4], dtype=int64)

In [27]:
model_pipeline.predict(["Hello"])

array([4], dtype=int64)

In [28]:
model_pipeline.predict(["fuck you"])

array([0], dtype=int64)

In [33]:
model_pipeline.predict(["hello, I really don't like this"])

array([0], dtype=int64)